In [1]:

import numpy

from hcve_lib.custom_types import ExceptionValue
from hcve_lib.utils import notebook_init

notebook_init()
from hcve_lib.evaluation_functions import average_group_scores
import pandas as pd
pd.options.plotting.backend = "plotly"
from mlflow import set_tracking_uri
import pandas
from deps.common import get_data_cached

set_tracking_uri('http://localhost:5000')

from plotly import express as px

from hcve_lib.tracking import load_group_results
%autoreload 2

GROUPS = {
    # 'stacking': '5e37865c-7821-4726-a015-56447f4b172d',
    'coxnet': '89be5f83-5679-42f2-ae11-f0788f8353d5',
    'gb': '52e729f7-0ae3-40a4-b639-8f9f3eaa3a36',
}

<frozen importlib._bootstrap>:228: RuntimeWarning: sklearn.tree._splitter.Splitter size changed, may indicate binary incompatibility. Expected 360 from C header, got 1160 from PyObject


In [2]:
data, metadata, X, y = get_data_cached()


[Memory]7.2s, 0.1min    : Loading get_data...
____________________________________________get_data cache loaded - 1.3s, 0.0min


In [3]:
from hcve_lib.utils import get_y_split
from deps.constants import RANDOM_STATE
from statistics import mean
from hcve_lib.functional import lagged

averaged_results = {}

for group_name, group_id in GROUPS.items():
    group = load_group_results(group_id, load_models=True)
    averaged_results[group_name] = average_group_scores(group)


In [4]:

from deps.logger import logger

for method, result in averaged_results.items():
    calibration_x = []
    calibration_y = []
    calibration_cohort = []
    calibration_x_label = []

    y_proba_per_split = []
    y_test_per_split = []
    for _, prediction in result.items():
        _, y_test = get_y_split(y, prediction)
        new_prediction = prediction['method'].predict(
            X=X,
            y=y,
            split=prediction['split'],
            model=prediction['model'],
            random_state=RANDOM_STATE,
            method=prediction['method'],
            time=y_test['data']['tte'],
        )
        y_proba = new_prediction['y_proba']['tte']
        y_proba_filtered = y_proba[y_proba.map(lambda i: not isinstance(i, ExceptionValue))]

        deleted = len(y_proba)-len(y_proba_filtered)
        if deleted > 0:
            logger.warning(f'Some TTE out of training range; n deleted = {deleted}')

        y_proba_per_split.append(y_proba_filtered)
        y_test_per_split.append(y_test['data'])

    y_proba_merged = pandas.concat(y_proba_per_split)
    y_test_merged = pandas.concat(y_test_per_split)

    quantiles = numpy.quantile(y_proba_merged, q=numpy.linspace(0, 1, 10))

    for quantile_from, quantile_to in lagged(quantiles):
        y_proba_subset = y_proba_merged[
            ((y_proba_merged >= quantile_from) &
             (y_proba_merged < quantile_to))
        ]
        y_subset = y['data'].loc[y_proba_subset.index]
        calibration_x.append(mean([quantile_from, quantile_to]))
        calibration_y.append(y_subset['label'].value_counts()[0] / len(y_subset['label']))
        calibration_x_label.append(f'{quantile_from}-{quantile_to}')

    fig = px.scatter(
        x=calibration_x,
        y=calibration_y,
        # range_x=[0,1], range_y=[0,1]
        width=600,
        height=600,
        title=method,
    )

    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )

    fig.show()


ValueError: Input X contains NaN.
DFCoxnetSurvivalAnalysisT does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:

from deps.logger import logger

for method, result in averaged_results.items():
    y_proba_per_split = []
    y_test_per_split = []
    for _, prediction in result.items():
        _, y_test = get_y_split(y, prediction)
        new_prediction = prediction['method'].predict(
            X=X,
            y=y,
            split=prediction['split'],
            model=prediction['model'],
            random_state=RANDOM_STATE,
            method=prediction['method'],
            time=y_test['data']['tte'],
        )
        y_proba = new_prediction['y_proba']['tte']
        y_proba_filtered = y_proba[y_proba.map(lambda i: not isinstance(i, ExceptionValue))]

        deleted = len(y_proba)-len(y_proba_filtered)
        if deleted > 0:
            logger.warning(f'Some TTE out of training range; n deleted = {deleted}')

        y_proba_per_split.append(y_proba_filtered)
        y_test_per_split.append(y_test['data'])

    y_proba_merged = pandas.concat(y_proba_per_split)
    y_test_merged = pandas.concat(y_test_per_split)

    quantiles = numpy.quantile(y_proba_merged, q=numpy.linspace(0, 1, 20))

    for quantile_from, quantile_to in lagged(quantiles):
        y_proba_subset = y_proba_merged[
            ((y_proba_merged >= quantile_from) &
             (y_proba_merged < quantile_to))
        ]
        y_subset = y['data'].loc[y_proba_subset.index]
        calibration_x.append(mean([quantile_from, quantile_to]))
        calibration_y.append(y_subset['label'].value_counts()[0] / len(y_subset['label']))
        calibration_x_label.append(f'{quantile_from}-{quantile_to}')

    fig = px.scatter(
        x=calibration_x,
        y=calibration_y,
        # range_x=[0,1], range_y=[0,1]
        width=400,
        height=400,
        title=method,
    )

    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )

    fig.show()
